# Title
[]()

In [3]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [4]:
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
# from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict, create_text_dict_from_folder
from wrangling import filter_df_all_conditions
from summary_chain import *

In [ ]:
# from response_processing import revive_chatbot, revive_chatbot_dict, save_instance_to_dict, process_chaining_results2

In [65]:
from response_processing import save_instance_to_dict, process_chaining_results2

In [4]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)

In [4]:
# from pypdf import PdfReader
import pandas as pd
import openai
import os
# from openai.embeddings_utils import get_embedding, cosine_similarity
import re

# Revive chatbot

## Load chatbot dict

In [12]:
filename = 'batch_Chaining_attributes_relevance_2023-05-02_1410.sav'
pickle_path = r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
chatbot_dict = loadpickle(filename, pickle_path)
chatbot_dict

Time completed: 2023-05-02 14:51:04.377476
Dictionary keys: ['text1_prompt00']


{'text1_prompt00': {'text': 'Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Similar Maximum Strength: Combining Cold Water Immersion and Compression\n\nDiscussion\nThe study aimed at investigating the effects of aging on muscle soreness and muscle damage following an intensive squat exercise protocol as well as on age-related effects of a recovery intervention in tightly controlled populations of young and master athletes regarding their absolute performance level. As an important experimental precondition, all markers of fatigue and recovery showed a significant within-subject effect of time of measurement with strikingly more pronounced changes from pre- to post-exercise (Figure 2), while CK peaked at post-24 in most individuals. Usually, the highest CK activity is found at 24 or 48h post-exercise. A mean CK increase of up to two times the baseline and up to 800U/l in some individuals can be attributed to structural damage to muscle cells 

In [15]:
chatbot_dict['text1_prompt00']['summaries_dict']

{'response_01': '"🏋️\u200d♀️ Muscle soreness and damage after intense exercise can affect athletes of all ages, according to a new study. 💪 Master athletes were found to recover just as quickly as their younger counterparts, contrary to previous beliefs. 🌊 However, a mixed-method recovery intervention (MMR) of cold water immersion and compression did not contribute to faster physical recovery, but did promote recovery of perceptual measures. 🤔 The effectiveness of MMR should be individualized and further studies are recommended. #fitness #exercise #recovery #athletes #MMR #science"',
 'response_02': 'Intensive squat exercises can lead to muscle soreness and damage, which affects recovery time. However, a study found that master athletes recover similarly to younger athletes despite aging. A mixed-method recovery intervention (MMR) did not significantly improve recovery of physical performance or muscle damage but did promote recovery of perceptual measures. It is recommended to individ

In [23]:
len(chatbot_dict['text1_prompt00']['simple_summary_dict'])
chatbot_dict['text1_prompt00']['simple_summary_dict']

{1: {'response_01': {0: {'simple summary choice': 1,
    'simplify task': 'Use language appropriate for',
    'audience': 'a lay audience',
    'full simplify task': 'Use language appropriate for for a lay audience',
    'simple summary': "A new study has found that intense exercise can make your muscles sore and cause damage, no matter how old you are. But the good news is that older athletes can recover just as quickly as younger ones, which is different from what we used to think. However, a treatment called mixed-method recovery (MMR), which involves using cold water and compression, didn't help people recover physically faster, but it did help them feel better mentally. It's important to remember that MMR might not work for everyone, and more research is needed. So if you're into fitness and exercise, keep this in mind!",
    'original summary': '"🏋️\u200d♀️ Muscle soreness and damage after intense exercise can affect athletes of all ages, according to a new study. 💪 Master athlet

## Update `revive_chatbot` function: Iteration 1.1

In [26]:
def revive_chatbot(chatbot):
    """
    Convert the dictionary of a chatbot attributes into a Chaining object.
    """
    new_chatbot = Chaining(chatbot['text'])
    print(f'Article title: {chatbot["article_title"]}')
    for key in chatbot.keys():
        setattr(new_chatbot, key, chatbot[key])
        print(f'\tNew chatbot attribute added: {key}')
        if type(getattr(new_chatbot, key)) == dict:
            print(f'\t\tAttribute dictionary keys: {[attr for attr in getattr(new_chatbot, key)]}')
    if not hasattr(chatbot[key], 'n_previous_prompts'):
        
        print(f'\tNew chatbot attribute added: n_previous_prompts')
        setattr(new_chatbot, 'n_previous_prompts',  dict())

    else:
        print(f'\tUpdating `n_previous_prompts`...')

    summaries_keys = [key for key in new_chatbot.summaries_dict.keys() if re.match(new_chatbot.response_regex, key)]
    print(f'\t\tPrevious number of prompts for summaries_dict: {len(summaries_keys)} {[key for key in summaries_keys]}')
    new_chatbot.n_previous_prompts['summaries'] = len(summaries_keys)
    new_chatbot.n_previous_prompts['simple_summary'] = len(new_chatbot.simple_summary_dict.keys())
    new_chatbot.n_previous_prompts['relevance'] = len(new_chatbot.relevance_dict.keys())

    print(f'\t\tPrevious number of prompts:')
    print(f'\t\t\tOriginal summaries: {len(summaries_keys)} {[key for key in summaries_keys]}')
    print(f'\t\t\tSimple summaries: {len(new_chatbot.simple_summary_dict.keys())}')
    print(f'\t\t\tAdded relevance: {len(new_chatbot.relevance_dict.keys())}')
    
    return new_chatbot


def revive_chatbot_dict(chatbot_dict, texts='all'):
    """
    Convert the dictionary of dictionaries of chatbot attributes into a dictionary of Chaining objects.

    Parameters:
        - chatbot_dict (dict): dictionary of dictionaries of chatbot attributes.
        - texts (list or 'all'): list of integers corresponding to the text prompts to revive. 
            If 'all', all text prompts will be revived.

    See "2023-05-01 test new prompts" notebook for example usage.
    """
    if texts == 'all':
        text_prompts_to_revive = [text for text in chatbot_dict.keys()]
    elif isinstance(texts, list):
        text_prompts_to_revive = [
            text_prompt for text_prompt in chatbot_dict.keys() for text in texts if f'{text}_' in text_prompt]
    else:
        raise TypeError("The `texts` parameter must be 'all' or a list of integers")

    new_chatbot_dict = {text_prompt: revive_chatbot(chatbot_dict[text_prompt]) for text_prompt in text_prompts_to_revive}
    print(f'\n\nNew chatbot dict keys: {[key for key in new_chatbot_dict]}')
    return new_chatbot_dict


new_chatbot_dict = revive_chatbot_dict(chatbot_dict, texts=[1])
new_chatbot_dict

Article title: Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Similar Maximum Strength: Combining Cold Water Immersion and Compression
	New chatbot attribute added: text
	New chatbot attribute added: system_role
	New chatbot attribute added: temperature
	New chatbot attribute added: max_tokens
	New chatbot attribute added: model
	New chatbot attribute added: qna
		Attribute dictionary keys: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary']
	New chatbot attribute added: summaries_dict
		Attribute dictionary keys: ['response_01', 'response_02', 'response_03', 'response_04', 'response_05', 'prep_step', 'task', 'prompt']
	New chatbot attribute added: article_title
	New chatbot attribute added: response_regex
	New chatbot attribute added: simple_summary_dict
		Attribute dictionary keys: [1]
	New chatbot attribute added: relevance_dict
		Attribute dictionary keys: [1, 2]
	New

{'text1_prompt00': <summary_chain.Chaining at 0x11d7fb53250>}

In [27]:
new_chatbot_dict['text1_prompt00'].n_previous_prompts

{'summaries': 5, 'simple_summary': 1, 'relevance': 2}

In [28]:
new_chatbot_dict

{'text1_prompt00': <summary_chain.Chaining at 0x11d7fb53250>}

# Save updated chatbot
## See if save filename will have original time stamp 
Code copied from `process_chaining_results2` function

In [31]:
original_summary_time = next(iter(new_chatbot_dict.values())).date_created
description = f'prompt_chain_summaries_and_relevance_{original_summary_time}_revived'
description
# save_instance_to_dict(
#     new_chatbot_dict,
#     description=description,
#     save_json=False
# )

'prompt_chain_summaries_and_relevance_2023-05-02_1410_revived'

In [34]:
save_instance_to_dict(
    new_chatbot_dict,
    description=description,
    save_json=False
)

text1_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
	date_created
	n_previous_prompts
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/prompt_chain_summaries_and_relevance_2023-05-02_1410_revived_2023-05-02_1524.sav
Time completed: 2023-05-02 15:24:00.546468
	Object saved as pickle
Dictionary keys: dict_keys(['text1_prompt00'])


{'text1_prompt00': {'text': 'Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Similar Maximum Strength: Combining Cold Water Immersion and Compression\n\nDiscussion\nThe study aimed at investigating the effects of aging on muscle soreness and muscle damage following an intensive squat exercise protocol as well as on age-related effects of a recovery intervention in tightly controlled populations of young and master athletes regarding their absolute performance level. As an important experimental precondition, all markers of fatigue and recovery showed a significant within-subject effect of time of measurement with strikingly more pronounced changes from pre- to post-exercise (Figure 2), while CK peaked at post-24 in most individuals. Usually, the highest CK activity is found at 24 or 48h post-exercise. A mean CK increase of up to two times the baseline and up to 800U/l in some individuals can be attributed to structural damage to muscle cells 

# Load saved instance

In [35]:
filename = 'prompt_chain_summaries_and_relevance_2023-05-02_1410_revived_2023-05-02_1524.sav'
pickle_path = r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
chatbot_dict2 = loadpickle(filename, pickle_path)


new_chatbot_dict2 = revive_chatbot_dict(chatbot_dict2)
new_chatbot_dict2

Time completed: 2023-05-02 15:27:13.882735
Dictionary keys: ['text1_prompt00']
Article title: Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Similar Maximum Strength: Combining Cold Water Immersion and Compression
	New chatbot attribute added: text
	New chatbot attribute added: system_role
	New chatbot attribute added: temperature
	New chatbot attribute added: max_tokens
	New chatbot attribute added: model
	New chatbot attribute added: qna
		Attribute dictionary keys: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary']
	New chatbot attribute added: summaries_dict
		Attribute dictionary keys: ['response_01', 'response_02', 'response_03', 'response_04', 'response_05', 'prep_step', 'task', 'prompt']
	New chatbot attribute added: article_title
	New chatbot attribute added: response_regex
	New chatbot attribute added: simple_summary_dict
		Attribute dictionary keys: [1]
	New c

{'text1_prompt00': <summary_chain.Chaining at 0x11d229c7640>}

In [39]:
vars(new_chatbot_dict2['text1_prompt00'])

{'text': 'Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Similar Maximum Strength: Combining Cold Water Immersion and Compression\n\nDiscussion\nThe study aimed at investigating the effects of aging on muscle soreness and muscle damage following an intensive squat exercise protocol as well as on age-related effects of a recovery intervention in tightly controlled populations of young and master athletes regarding their absolute performance level. As an important experimental precondition, all markers of fatigue and recovery showed a significant within-subject effect of time of measurement with strikingly more pronounced changes from pre- to post-exercise (Figure 2), while CK peaked at post-24 in most individuals. Usually, the highest CK activity is found at 24 or 48h post-exercise. A mean CK increase of up to two times the baseline and up to 800U/l in some individuals can be attributed to structural damage to muscle cells (Baird et al., 2012

# Revive chatbot after `Chaining` scripts modified to add `n_previous_prompts` attribute when instance is created

In [37]:
filename = 'batch_Chaining_attributes_relevance_2023-05-02_1820.sav'
pickle_path = r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
chatbot_dict2 = loadpickle(filename, pickle_path)
chatbot_dict2

Time completed: 2023-05-02 18:27:59.542836
Dictionary keys: ['text1_prompt00']


{'text1_prompt00': {'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength 

In [38]:
new_chatbot2 = revive_chatbot_dict(chatbot_dict2)
new_chatbot2

Article title: Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men
	New chatbot attribute added: text
	New chatbot attribute added: system_role
	New chatbot attribute added: temperature
	New chatbot attribute added: max_tokens
	New chatbot attribute added: model
	New chatbot attribute added: qna
		Attribute dictionary keys: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary']
	New chatbot attribute added: summaries_dict
		Attribute dictionary keys: ['response_01', 'response_02', 'response_03', 'response_04', 'response_05', 'prep_step', 'task', 'prompt']
	New chatbot attribute added: article_title
	New chatbot attribute added: response_regex
	New chatbot attribute added: simple_summary_dict
		Attribute dictionary keys: [1]
	New chatbot attribute added: relevance_dict
		Attribute dictionary keys: [1]
	New chatbot attribute added: n_previous_prompts
		Attribute dictionary keys

{'text1_prompt00': <summary_chain.Chaining at 0x11d229c7bb0>}

# Send new prompts

## Tasks

In [43]:
prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

summarize_task = [
    # "Summarize for a LinkedIn post",
    # "Describe the interesting points to your coworker at the water cooler",
    # "Create a Tiktok post",
    "Create an Instagram post",
    # "Summarize the article for a Tiktok post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    # "Summarize the article in under 300 characters",
]

simple_simplify_task = [
    # "Use terms a 12-year-old can understand.",
    "Assume your audience has no science background.",
    # "Use a fun tone."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

user_simplify_task = [
    # "Use language appropriate for",
    "If needed, rewrite using terms appropriate for the audience. If not needed, return nothing. Your audience is",
    # "Use terms a 12-year-old can understand.",
]

simplify_audience = [
    # "",
    "a lay audience",
    # "people without a science background",
]

user_relevance_task = [
    "Rewrite the summary to include an explanation of how it is relevant for",
    # "Add 1-2 sentences to make this relevant for",
    # "Update the summary with 1-2 sentences to make it relevant for",
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
]

relevance_audience = [
    # "lay audience",
    # "",
    # "seniors",
    # "people who enjoy sports",
    "people new to resistance training"
]

## Simplify

In [48]:
print(user_simplify_task)
print(simplify_audience)

['If needed, rewrite using terms appropriate for the audience. If not needed, return nothing. Your audience is']
['a lay audience']


In [50]:
iteration_id = 2

# prompt_column = 'relevance'
# prompts = user_relevance_task
# relevance_dict = dict()
# audience = relevance_audience

prompts = user_simplify_task
simplify_dict = dict()
chaining_dict = dict()
chaining_dict[iteration_id] = new_chatbot_dict
audience = simplify_audience
prompt_column = 'simplify'
results_dict = simplify_dict

pause_per_request = 0
summary_iteration_id = iteration_id

results_dict = prompt_chaining_dict(prompts, audience, results_dict, 
    chaining_dict[summary_iteration_id], iteration_id, prompt_column=prompt_column,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

summary_iteration_id: 2
n_choices: 1
**text1_prompt00
simplify_iteration:  2
Task: If needed, rewrite using terms appropriate for the audience. If not needed, return nothing. Your audience is a lay audience
summaries_keys: 
	 ['response_01', 'response_02', 'response_03', 'response_04', 'response_05']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_04
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GP

In [64]:
print(len(results_dict[iteration_id][0]))
results_dict[iteration_id][0]

2


{1: {'response_01': {0: {'simple summary choice': 1,
    'simplify task': 'Use language appropriate for',
    'audience': 'a lay audience',
    'full simplify task': 'Use language appropriate for for a lay audience',
    'simple summary': "A new study has found that intense exercise can make your muscles sore and cause damage, no matter how old you are. But the good news is that older athletes can recover just as quickly as younger ones, which is different from what we used to think. However, a treatment called mixed-method recovery (MMR), which involves using cold water and compression, didn't help people recover physically faster, but it did help them feel better mentally. It's important to remember that MMR might not work for everyone, and more research is needed. So if you're into fitness and exercise, keep this in mind!",
    'original summary': '"🏋️\u200d♀️ Muscle soreness and damage after intense exercise can affect athletes of all ages, according to a new study. 💪 Master athlet

In [70]:
pd.DataFrame(chaining_dict[iteration_id]['text1_prompt00'].qna)

,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary
0,Recovery From Eccentric Squat Exercise in Resi...,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,"""🏋️‍♀️ Muscle soreness and damage after intens..."
1,Recovery From Eccentric Squat Exercise in Resi...,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,Intensive squat exercises can lead to muscle s...
2,Recovery From Eccentric Squat Exercise in Resi...,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,Recovery after intense exercise can be a chall...
3,Recovery From Eccentric Squat Exercise in Resi...,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,Recovery after intense exercise is crucial for...
4,Recovery From Eccentric Squat Exercise in Resi...,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,🔬 SCIENCE NEWS: A recent study investigated th...


# `process_chaining_results3`

In [67]:
help(process_chaining_results2)

Help on function process_chaining_results2 in module response_processing:

process_chaining_results2(chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple', empty_columns=None, pivot=True, validate=None, chatbot_id=None, save_df=False, save_chatbot=False, csv_path='C:\\Users\\silvh\\OneDrive\\lighthouse\\Ginkgo coding\\content-summarization\\output', pickle_path='C:\\Users\\silvh\\OneDrive\\lighthouse\\Ginkgo coding\\content-summarization\\output\\pickles', json_path='C:\\Users\\silvh\\OneDrive\\lighthouse\\Ginkgo coding\\content-summarization\\output\\json')
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    
    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (di

In [104]:
import string
def process_chaining_results3(
        chain_results_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_df.columns

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - results_type (str): 'simple' or 'relevance'. Default is 'simple'.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.

    """
    df_list = []
    qna_dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts[results_type]
            print(f'\tNumber of previous {results_type} prompts: {n_previous_prompts}')
        except:
            n_previous_prompts = 0
            print(f'No previous {results_type} prompts for {chatbot_key}')
            
        qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna))

        if results_type=='simple':
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
            chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number]
        for iteration_key in results_dict.keys():
            response_keys = sorted([text_prompt_key for text_prompt_key in results_dict[iteration_key].keys()])
            print(f'\tAppending results for {iteration_key}: ', end='')
            for response_key in response_keys:
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
                print(f'{response_key}, ', end='')
            print('')

    
    new_results = pd.concat(df_list)
    qna_df = pd.concat(qna_dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    print('New results shape:', new_results.shape)
    
    original_summary_columns = [
        'article_title',
        # 'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    pivot = False if results_type=='simple' else pivot
    if (pivot == False) or (results_type=='simple'):
        new_results = qna_df[original_summary_columns].merge(
            new_results, how='right', 
            right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
            left_on='summary'
        )
        if results_type=='simple':
            columns = [
                'article_title',
                # 'choice',
                'system_role',
                'model',
                'text',
                'prep step',
                'summarization task',
                'full summarization task',
                'summary',
                'simple summary choice',
                'audience',
                'simplify task',
                'full simplify task',
                'simple summary'
            ]
        else:
            columns= [
                'article_title',
                'choice',
                'system_role',
                'model',
                'text',
                'prep step',
                'summarization task',
                'full summarization task',
                'summary',
                'relevance choice',
                'audience',
                'relevance task',
                'full relevance task',
                'relevance statement'
            ]
        print(f'New results columns: {new_results.columns}')
        new_results = new_results[columns]
    else:
        new_results_pivot_df = new_results.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task']
        ).reset_index()
        print(f'Shape of pivoted dataframe: {new_results_pivot_df.shape}')
        new_results = qna_df[original_summary_columns].merge(
            new_results_pivot_df, how='outer', suffixes=(' original', ' relevance'),
            left_on='summary',
            right_on=f'{"original" if results_type=="simple" else "preceding"} summary',
            validate='m:1' if validate else None
        ).drop(columns=['preceding summary'])
        print(f'Merged DataFrame shape: {new_results.shape}')
        new_results.drop_duplicates(inplace=True)
        print(f'Duplicate rows dropped. Shape: {new_results.shape}')

    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                if results_type=='simple':
                    empty_columns = {
                        "choice numnber": "C",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                        # "simple summary choice": "N",
                    }
                else:
                    empty_columns = {
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                        # "simple summary choice": "N",
                        "simplify audience": "O",
                        "simplify task": "P",
                        "full simplify task": "Q",
                        "simple summary": "R",
                        "simple summary content rating": "S",
                        "simple summary language rating": "T",
                        "top simple summary": "U",
                    }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                        "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    'add relevance task (seniors)': 'AB',
                    'full add relevance task (seniors)': 'AC',
                }
        print(f'Merged DataFrame shape: {new_results.shape}')
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

iteration_id = 2
chain_results_dict = {}
chatbot_dict = {}
save = False

chain_results_dict = process_chaining_results3(
    chain_results_dict, chaining_dict, iteration_id, results_type = 'simple',
    empty_columns=1, save_chatbot=save
)


Processing text1_prompt00...
No previous simple prompts for text1_prompt00
	Appending results for 1: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 2: response_01, response_02, response_03, response_04, response_05, 
Original summaries DataFrame shape: (5, 8)
Original summaries Dataframe columns: Index(['article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dtype='object')
New results shape: (10, 6)
New results columns: Index(['article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary', 'audience',
       'full simplify task', 'original summary', 'simple summary',
       'simple summary choice', 'simplify task'],
      dtype='object')
Merged DataFrame shape: (10, 13)

Columns before adding empty columns: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summ

In [105]:
chain_results_dict[iteration_id]

,B: article_title,C: choice numnber,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,L: original summary language rating,M: top summary,N: simple summary choice,O: audience,P: simplify task,Q: full simplify task,R: simple summary
0,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,"""🏋️‍♀️ Muscle soreness and damage after intens...",,,,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A new study has found that intense exercise ca...
1,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,Intensive squat exercises can lead to muscle s...,,,,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"If you do a lot of squat exercises, your muscl..."
2,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,Recovery after intense exercise can be a chall...,,,,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"After a tough workout, it can be hard for athl..."
3,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,Recovery after intense exercise is crucial for...,,,,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,Recovering after a tough workout is really imp...
4,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,🔬 SCIENCE NEWS: A recent study investigated th...,,,,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,Scientists have recently studied how aging aff...
5,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,"""🏋️‍♀️ Muscle soreness and damage after intens...",,,,1,a lay audience,"If needed, rewrite using terms appropriate for...","If needed, rewrite using terms appropriate for...",A new study found that muscle soreness and dam...
6,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,Intensive squat exercises can lead to muscle s...,,,,1,a lay audience,"If needed, rewrite using terms appropriate for...","If needed, rewrite using terms appropriate for...",Intense squat exercises can make your muscles ...
7,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take t

## Iteration 1.2: make sure it works for relevance results

In [111]:
import string
def process_chaining_results3(
        chain_results_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Create a dataframe of new 'simple' or 'relevance' summaries from a Chaining object.
    Merge it with the original summaries DataFrame.

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - results_type (str): 'simple' or 'relevance'. Default is 'simple'.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.

    """
    df_list = []
    qna_dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts[results_type]
            print(f'\tNumber of previous {results_type} prompts: {n_previous_prompts}')
        except:
            n_previous_prompts = 0
            print(f'No previous {results_type} prompts for {chatbot_key}')
            
        qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna))

        if results_type=='simple':
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
            chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number]
        for iteration_key in results_dict.keys():
            response_keys = sorted([text_prompt_key for text_prompt_key in results_dict[iteration_key].keys()])
            print(f'\tAppending results for {iteration_key}: ', end='')
            for response_key in response_keys:
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
                print(f'{response_key}, ', end='')
            print('')

    
    new_results = pd.concat(df_list)
    qna_df = pd.concat(qna_dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    print('New results shape:', new_results.shape)
    
    original_summary_columns = [
        'article_title',
        # 'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    pivot = False if results_type=='simple' else pivot
    if (pivot == False) or (results_type=='simple'):
        new_results = qna_df[original_summary_columns].merge(
            new_results, how='right', 
            right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
            left_on='summary'
        )
        if results_type=='simple':
            columns = [
                'article_title',
                # 'choice',
                'system_role',
                'model',
                'text',
                'prep step',
                'summarization task',
                'full summarization task',
                'summary',
                'simple summary choice',
                'audience',
                'simplify task',
                'full simplify task',
                'simple summary'
            ]
        else:
            columns= [
                'article_title',
                'choice',
                'system_role',
                'model',
                'text',
                'prep step',
                'summarization task',
                'full summarization task',
                'summary',
                'relevance choice',
                'audience',
                'relevance task',
                'full relevance task',
                'relevance statement'
            ]
        print(f'New results columns: {new_results.columns}')
        new_results = new_results[columns]
    else:
        new_results_pivot_df = new_results.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task']
        ).reset_index()
        print(f'Shape of pivoted dataframe: {new_results_pivot_df.shape}')
        new_results = qna_df[original_summary_columns].merge(
            new_results_pivot_df, how='outer', suffixes=(' original', ' relevance'),
            left_on='summary',
            right_on=f'{"original" if results_type=="simple" else "preceding"} summary',
            validate='m:1' if validate else None
        ).drop(columns=['preceding summary'])
        print(f'Merged DataFrame shape: {new_results.shape}')
        new_results.drop_duplicates(inplace=True)
        print(f'Duplicate rows dropped. Shape: {new_results.shape}')

    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                if results_type=='simple':
                    empty_columns = {
                        "choice numnber": "C",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                    }
                else:
                    empty_columns = {
                        "choice numnber": "C",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                        "simplify audience": "O",
                        "simplify task": "P",
                        "full simplify task": "Q",
                        "simple summary": "R",
                        "simple summary content rating": "S",
                        "simple summary language rating": "T",
                        "top simple summary": "U",
                    }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                    "choice numnber": "C",
                        "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    'add relevance task (seniors)': 'AB',
                    'full add relevance task (seniors)': 'AC',
                }
        print(f'Merged DataFrame shape: {new_results.shape}')
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

# iteration_id = 2
# chain_results_dict = {}
# chatbot_dict = {}
# save = False

# chain_results_dict = process_chaining_results3(
#     chain_results_dict, chaining_dict, iteration_id, results_type = 'simple',
#     empty_columns=1, save_chatbot=save
# )

iteration_id = 2
relevance_results_dict = {}
save = False

relevance_results_dict = process_chaining_results3(
    relevance_results_dict, chaining_dict, iteration_id, results_type='relevance',
    empty_columns=1, save_chatbot=save
)



Processing text1_prompt00...
No previous relevance prompts for text1_prompt00
	Appending results for 1: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 2: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 3: response_01, response_02, response_03, response_04, 
Original summaries DataFrame shape: (5, 8)
Original summaries Dataframe columns: Index(['article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dtype='object')
New results shape: (14, 6)
Shape of pivoted dataframe: (5, 5)
Merged DataFrame shape: (5, 12)
Duplicate rows dropped. Shape: (5, 12)
Merged DataFrame shape: (5, 12)

Columns before adding empty columns: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance task', 'people new to resistance training', 'people who enjoy sports', 'seniors']


In [113]:
relevance_results_dict[iteration_id]

,B: article_title,C: choice numnber,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,...,V: relevance task,W: full add relevance task,X: people new to resistance training,Y: added relevance content rating,Z: added relevance language rating,AA: top added relevance,AB: add relevance task (seniors),AC: full add relevance task (seniors),AD: people who enjoy sports,AE: seniors
0,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,"""🏋️‍♀️ Muscle soreness and damage after intens...",,...,Rewrite the summary to include an explanation ...,,A recent study found that muscle soreness and ...,,,,,,A recent study has found that muscle soreness ...,"According to a recent study, muscle soreness a..."
1,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,Intensive squat exercises can lead to muscle s...,,...,Rewrite the summary to include an explanation ...,,"For those new to resistance training, it's imp...",,,,,,"For sports enthusiasts, it is important to und...",Intensive squat exercises can cause muscle sor...
2,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,Recovery after intense exercise can be a chall...,,...,Rewrite the summary to include an explanation ...,,"For individuals new to resistance training, re...",,,,,,"For sports enthusiasts, recovering after inten...",Recovery after intense exercise can be challen...
3,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,Recovery after intense exercise is crucial for...,,...,Rewrite the summary to include an explanation ...,,Recovery is a critical aspect of exercise for ...,,,,,,"For sports enthusiasts, recovery after intense...",Recovery is a critical aspect of exercise for ...
4,Recovery From Eccentric Squat Exercise in Resi...,,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Create an Instagram post,Take the key points and numerical descriptors ...,🔬 SCIENCE NEWS: A recent study investigated th...,,...,Rewrite the summary to include an explanation ...,,NaN,,,,,,🔬 SPORTS NEWS: If you're an athlete who loves ...,🔬 SCIENCE NEWS: A recent study investigated th...


# *End of Page*